# TFLite API 서버 배포

### 1단계 - 모델을 .tflite 형식으로 변환

In [2]:
import tensorflow as tf

# 기존 Keras 모델 로드
model = tf.keras.models.load_model('acne_classifier.keras')

# TFLite Converter 사용
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# TFLite 파일로 저장
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ TFLite 모델 변환 및 저장 완료: model.tflite")

INFO:tensorflow:Assets written to: /var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg/assets


INFO:tensorflow:Assets written to: /var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg/assets


Saved artifact at '/var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  5972853584: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  5972867600: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  6037662864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037663632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037660560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037662096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037661520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037662672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037661136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6037662288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  603

W0000 00:00:1764169640.021393  158174 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1764169640.021913  158174 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-11-27 00:07:20.023317: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg
2025-11-27 00:07:20.032473: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-11-27 00:07:20.032487: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg
2025-11-27 00:07:20.147119: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-11-27 00:07:20.716091: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/18/0c_kq_wd7qqd26whwky82_nc0000gn/T/tmpj71p8rxg
2025-11-27 00:07:20.885306: I tensorflow/cc/saved_model/loader.cc:

### 2단계 - FastAPI로 TFLite API 서버 구성

In [7]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import numpy as np
import tensorflow as tf
import cv2

app = FastAPI()

# 클래스 이름 매핑 (필요 시 사용자 정의)
CLASS_NAMES = ["Normal", "Comedones", "Pustules", "Papules", "Folliculitis"]

# TFLite Interpreter 로드
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
INPUT_SHAPE = input_details[0]['shape'][1:3]

def preprocess_image(image_bytes):
    img_array = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError("Invalid image data")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, tuple(INPUT_SHAPE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0).astype(np.float32)
    return img

@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        img = preprocess_image(contents)

        interpreter.set_tensor(input_details[0]['index'], img)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])[0]

        pred_class_idx = int(np.argmax(output_data))
        pred_class_name = CLASS_NAMES[pred_class_idx]
        confidence = float(np.max(output_data))

        return JSONResponse({
            "prediction_index": pred_class_idx,
            "prediction_label": pred_class_name,
            "confidence": confidence,
            "scores": output_data.tolist()
        })

    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Prediction failed: {str(e)}")